# Welkom bij Tutorial 4

Dit is alweer de laatste tutorial van Algoritmiek en Datastructuren, waarin wij **grafen** gaan bestuderen. Een graaf is een datastructuur waarin nodes (die in de context van grafen ook wel **vertices** heten) aan elkaar verbonden zijn door middel van **edges**. 

De manier waarop er meestal over grafen geschreven wordt is al een tuple $G = (V, E)$ waarbij G de graaf is, V de *set* van nodes/vertices en E de *set* van edges. Ik schrijf hier *set* schuingedrukt, omdat dit een bekende datastructuur voor ons is: een container waarin geen dubbelen mogen zitten. 

Vertices zijn uiteraard een classes die data kunnen bevatten, maar ook de edges kunnen soms data bevatten. Bijvoorbeeld, in een shortest path problem (zie Reader) stellen de vertices/nodes locaties voor, en bevatten dus de naam van de locatie, en de edges verbindingen tussen deze locaties. De edges bevatten de *afstand* (bijvoorbeeld in kilometers) tussen deze locaties. 

Grafen zijn een zeer flexibele manier van data structuren: zolang er maar verbindingen tussen nodes en edges gelegd kunnen worden is het een valide implementatie van een graaf. Dit kan je natuurlijk op heel veel verschillende manieren doen. Laten we daar eens een paar van gaan bekijken. 

We gaan dit graafje maken: https://en.wikipedia.org/wiki/File:Dijkstra_Animation.gif 

In [189]:
from matplotlib import pyplot as plt
import numpy as np
import time
import statistics as stats

In [190]:
# Een zeer generieke manier om een graaf te implementeren is er
# daarwerkelijk twee sets van te maken op basis van twee classes:
class Vertex:
    def __init__(self, identifier, data_):
        self.id = identifier
        self.data = data_

    def __eq__(self, other):  # nodig om aan een set toe te voegen
        return self.id == other.id

    def __hash__(self):  # nodig om aan een set toe te voegen
        return hash(self.id)

    def __repr__(self):
        return str(self.id) + ":" + str(self.data)


class Edge:
    def __init__(self, vertex1, vertex2, data_):
        if vertex1.id < vertex2.id:
            self.v1 = vertex1
            self.v2 = vertex2
        else:
            self.v1 = vertex2
            self.v2 = vertex1
        self.data = data_

    def __eq__(self, other):  # nodig om aan een set toe te voegen
        return self.v1.id == other.v1.id and self.v2.id == self.v2.id

    def __hash__(self):  # nodig om aan een set toe te voegen
        return hash(str(self.v1.id) + "," + str(self.v2.id))

    def __repr__(self):
        return "(" + str(self.v1.id) + "," + str(self.v2.id) + "):" + str(self.data)


class CGraph:
    def __init__(self):
        self.V = set()
        self.E = set()

    def __str__(self):
        return "V: " + str(self.V) + "\nE: " + str(self.E)


# So, for a simple shortest path problem:
gr = CGraph()
v1 = Vertex(1, "start")
v2 = Vertex(2, "")
v3 = Vertex(3, "")
v4 = Vertex(4, "")
v5 = Vertex(5, "goal")
v6 = Vertex(6, "")
gr.V.add(v1)
gr.V.add(v2)
gr.V.add(v3)
gr.V.add(v4)
gr.V.add(v5)
gr.V.add(v6)
e1 = Edge(v1, v2, 7)
e2 = Edge(v1, v3, 9)
e3 = Edge(v1, v6, 14)
e4 = Edge(v2, v3, 10)
e5 = Edge(v2, v4, 15)
e6 = Edge(v3, v4, 11)
e7 = Edge(v3, v6, 2)
e8 = Edge(v6, v5, 9)
e9 = Edge(v4, v5, 6)
gr.E.add(e1)
gr.E.add(e2)
gr.E.add(e3)
gr.E.add(e4)
gr.E.add(e5)
gr.E.add(e6)
gr.E.add(e7)
gr.E.add(e8)
gr.E.add(e9)
print(gr)

# Dit is eigenlijk al best een langdradige manier van doen...

V: {1:start, 2:, 3:, 4:, 5:goal, 6:}
E: {(1,2):7, (1,6):14, (3,4):11, (3,6):2, (5,6):9, (4,5):6, (2,4):15, (1,3):9, (2,3):10}


In [191]:
# Daarom kiezen mensen er vaak voor om een graaf makkelijker te respresenteren met standaard datatypes
# bijvoorbeeld: als we een dictionary maken met node identifiers als keys, en een tuple van (data, edges) als values
# met edges als een dictionary met vertex ids van de verbonden vertices als keys en de data van de edge als key
# krijgen we zoiets voor dezelfde graaf
DGraph = dict
gr2 = {
    1: ("start", {2: 7, 3: 9, 6: 14}),
    2: ("", {1: 7, 3: 10, 4: 15}),
    3: ("", {1: 9, 2: 10, 4: 11, 6: 2}),
    4: ("", {2: 15, 3: 11, 5: 6}),
    5: ("goal", {4: 6, 6: 9}),
    6: ("", {1: 14, 3: 2, 5: 9}),
}
print(gr2)
# Dat is makkelijker in te voeren... Maar je moet wel goed bijhouden wat ookalweer wat is.
# Bovendien heb ik er nu wat redundantie ingezet (elke node bevat de edges waaraan het verbonden zit, dus elke
# edge zit twee maal in deze datastructuur)

{1: ('start', {2: 7, 3: 9, 6: 14}), 2: ('', {1: 7, 3: 10, 4: 15}), 3: ('', {1: 9, 2: 10, 4: 11, 6: 2}), 4: ('', {2: 15, 3: 11, 5: 6}), 5: ('goal', {4: 6, 6: 9}), 6: ('', {1: 14, 3: 2, 5: 9})}


## Opgave 1: Dijkstra

Laten we eens gaan kijken hoe goed de twee mogelijke implementaties van een graaf hierboven zijn (dus die met de expliciete klasses voor Vertices en Edges, en die die de hele graaf in één dictionary gooit. 

Schrijf implementaties van Dijkstra's algoritme voor de beide implementaties van grafen (CGraph en DGraph). Vergelijk de snelheid van beide versies van Dijkstra's algoritme door beiden meerdere keren te runnen. Vergeet niet de standaardafwijkingen te berekenen. Is één implementatie beter dan de andere? 

In [192]:
# Schrijf hier de code voor opgave 1
def shortestPath_CGraph(graph, start, finish):
    for n in graph.V:
        n.dist   = float('inf')
        n.prev   = None 
    start.dist = 0
    
    N = {start}
    S = set()
    while len(N) != 0:
        n = min(N, key=lambda x: x.dist)
        N.remove(n)
        S.add(n)
        
        if n == finish:
            break
        
        for edge in graph.E:
            if n == edge.v1:
                m = edge.v2
            elif n == edge.v2:
                m = edge.v1
            else:
                continue

            if m not in S:
                if m not in N:
                    N.add(m)
                
                altDistance = n.dist + edge.data
                if m.dist > altDistance:
                    m.dist = altDistance
                    m.prev = n
                    
    path = []
    current_node = finish
    while current_node:
        path.append(current_node.id)
        current_node = current_node.prev
    path.reverse()

    return finish.dist, path

print(shortestPath_CGraph(gr, v1, v5))

(20, [1, 3, 6, 5])


In [193]:
def shortestPath_DGraph(graph, start, finish):
    dist = {}
    prev  = {}
    
    for n in graph.keys():
        dist[n] = float('inf')
        prev[n]    = None
    dist[start] = 0
  
    N = {start}
    S = set()
    while len(N) != 0:
        n = min(N, key=lambda x: dist[x])
        N.remove(n)
        S.add(n)
        
        if n == finish:
            break
        
        for m, data in graph[n][1].items():
            if m not in S:
                if m not in N:
                    N.add(m)

                altDistance = dist[n] + data
                if dist[m] > altDistance:
                    dist[m] = altDistance
                    prev[m] = n
                    
    path = []
    current_node = finish
    while current_node:
        path.append(current_node)
        current_node = prev[current_node]
    path.reverse()

    return dist[finish], path

print(shortestPath_DGraph(gr2, 1, 5))

(20, [1, 3, 6, 5])


*Schrijf hier de tekst-antwoorden voor Opgave 1*

## Opgave 2: Een andere implentatie?

Verzin (evt. samen met je teamleden) nog een andere manier om grafen te implementeren, en schrijf vervolgens weer een bijbehorende implementatie van Dijkstra's algoritme. Is deze beter dan de vorige twee manieren? Toon dit wederom aan door metingen te doen. 

Ter inspiratie: kijk eens naar **adjacency lists** https://en.wikipedia.org/wiki/Adjacency_list en **adjacency matrices** https://en.wikipedia.org/wiki/Adjacency_matrix . Zou je de edges misschien (evt. per vertex) kunnen sorteren op een manier dat Dijkstra's algoritme makkelijker maakt, en zo ja, is dat computationeel de moeite waard?

In [195]:
# Schrijf hier de code voor opgave 2
AMGraph = np.array([
    [0 , 7 , 9 , 0 , 0 , 14],
    [7 , 0 , 10, 15, 0 , 0 ],
    [9 , 10, 0 , 11, 0 , 2 ],
    [0 , 15, 11, 0 , 6 , 0 ],
    [0 , 0 , 0 , 6 , 0 , 9 ],
    [14, 0 , 2 , 0 , 9 , 0 ]
])

def shortestPath_AMGraph(graph, start, finish):
    dist = {}
    prev  = {}
    
    for n in range(len(graph)):
        dist[n] = float('inf')
        prev[n]    = None
    dist[start] = 0
  
    N = {start}
    S = set()
    while len(N) != 0:
        n = min(N, key=lambda x: dist[x])
        N.remove(n)
        S.add(n)
        
        if n == finish:
            break
        
        for m, data in graph[n][1].items():
            if m not in S:
                if m not in N:
                    N.add(m)

                altDistance = dist[n] + data
                if dist[m] > altDistance:
                    dist[m] = altDistance
                    prev[m] = n
                    
    path = []
    current_node = finish
    while current_node:
        path.append(current_node)
        current_node = prev[current_node]
    path.reverse()

    return dist[finish], path

print(shortestPath_AMGraph(AMGraph, 0, 4))

AttributeError: 'numpy.int32' object has no attribute 'items'

*Schrijf hier de tekst-antwoorden voor Opgave 2*